<a href="https://colab.research.google.com/github/cawcawlabs/ouster_lidar_appnotes/blob/main/google_colab_demos/pointpillar_osx.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install open3d
!git clone https://github.com/isl-org/Open3D-ML.git


# To install a compatible version of PyTorch with CUDA
%cd /content/Open3D-ML
!pip install -r requirements-torch-cuda.txt
%cd /content/

!git clone https://github.com/cawcawlabs/ouster_lidar_appnotes.git
!pip install ouster-sdk


!wget https://storage.googleapis.com/data.ouster.io/concept-engineering/ptpillar/pointpillars_kitti.yml
#REV6 urban drive sample, calibrated reflectivity
!wget https://storage.googleapis.com/data.ouster.io/concept-engineering/ctp/OS-1-128_v2.3.0_1024x10_20220419_160551-000.pcap
!wget https://storage.googleapis.com/data.ouster.io/concept-engineering/ctp/OS-1-128_v2.3.0_1024x10_20220419_160551.json

!wget https://storage.googleapis.com/data.ouster.io/concept-engineering/ptpillar/pointpillars_kitti_202012221652utc.pth


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 408.6 MB 30 kB/s 
     |████████████████████████████████| 1.1 MB 49.5 MB/s 
     |████████████████████████████████| 596 kB 65.0 MB/s 
     |████████████████████████████████| 3.1 MB 49.4 MB/s 
     |████████████████████████████████| 8.8 MB 48.0 MB/s 
     |████████████████████████████████| 428 kB 47.4 MB/s 
     |████████████████████████████████| 54 kB 2.7 MB/s 
     |████████████████████████████████| 344 kB 71.8 MB/s 
     |████████████████████████████████| 1.2 MB 50.7 MB/s 
     |████████████████████████████████| 54 kB 3.0 MB/s 
     |████████████████████████████████| 561 kB 67.7 MB/s 
     |████████████████████████████████| 80 kB 10.6 MB/s 
     |████████████████████████████████| 132 kB 74.8 MB/s 
     |████████████████████████████████| 133 kB 61.9 MB/s 
     |████████████████████████████████| 106 kB 66.4 MB/s 
  Created wheel for json5: filename=

Cloning into 'Open3D-ML'...
remote: Enumerating objects: 6734, done.
remote: Counting objects: 100% (690/690), done.
remote: Compressing objects: 100% (278/278), done.
remote: Total 6734 (delta 459), reused 582 (delta 394), pack-reused 6044
Receiving objects: 100% (6734/6734), 37.72 MiB | 13.27 MiB/s, done.
Resolving deltas: 100% (4547/4547), done.
/content/Open3D-ML
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://download.pytorch.org/whl/lts/1.8/torch_lts.html
Ignoring torch: markers 'python_version == "3.6"' don't match your environment
     |█████████████████████▎          | 834.1 MB 2.1 MB/s eta 0:03:23tcmalloc: large alloc 1147494400 bytes == 0x28c8000 @  0x7f348ec69615 0x592b76 0x4df71e 0x59afff 0x515655 0x549576 0x593fce 0x548ae9 0x51566f 0x549576 0x593fce 0x548ae9 0x5127f1 0x598e3b 0x511f68 0x598e3b 0x511f68 0x598e3b 0x511f68 0x4bc98a 0x532e76 0x594b72 0x515600 0x549576 0x593fce 0x548ae9 0x5127f1 0x549

In [3]:
import os
import open3d.ml as _ml3d
import open3d.ml.torch as ml3d
from open3d.ml.vis import Visualizer, BoundingBox3D, LabelLUT
import sys
from pathlib import Path
import pprint
import numpy as np

from ouster import client, pcap
from contextlib import closing
from more_itertools import nth

pcap_path = '/content/OS-1-128_v2.3.0_1024x10_20220419_160551-000.pcap'
metadata_path = '/content/OS-1-128_v2.3.0_1024x10_20220419_160551.json'

with open(metadata_path, 'r') as f:
        metadata = client.SensorInfo(f.read())

source = pcap.Pcap(pcap_path, metadata)

with closing(client.Scans(source)) as scans:
        scan = nth(scans, 1)

xyzlut = client.XYZLut(source.metadata)

#destagger
signal  = client.destagger(metadata,scan.field(client.ChanField.REFLECTIVITY))
#signal = (signal / np.max(signal)).astype(np.float32) no longer needed for calibrated reflectivity
xyz_destaggered = client.destagger(metadata, xyzlut(scan))

#print(intensity_destaggered[0,2047]) #128 X 2048
#print(xyz_destaggered[0,2047]) #128 x 2048

[x, y, z] = [c.flatten() for c in np.dsplit(xyz_destaggered, 3)]
signal = signal.flatten()

point_cloud = np.asarray([x,y,z,signal], dtype =  np.float32).transpose()


#AND NOW TO PT_PILLAR
cfg_file = "/content/pointpillars_kitti.yml"

cfg = _ml3d.utils.Config.load_from_file(cfg_file)

model = ml3d.models.PointPillars(**cfg.model)

pipeline = ml3d.pipelines.ObjectDetection(model, device="gpu", **cfg.pipeline)

ckpt_path = '/content/pointpillars_kitti_202012221652utc.pth'

# load the parameters.
pipeline.load_ckpt(ckpt_path=ckpt_path)

data = {
            'point': point_cloud,
            'full_point': point_cloud,
            'feat': None,
            'calib': None,
            'bounding_boxes': [],
        }


result = pipeline.run_inference(data)

print(result)

#print(data)
#boxes = data['bounding_boxes']
#boxes.extend(result)

#print(boxes)

#(pos,dim,yaw,name,score,world_cam, cam_img)

# box_list = result[0]
# bbox_list=[]
# for box in box_list:
#     pos = box[0]
#     dim = box[1]
#     yaw = box[2]
#     name = box[3]
#     score = box[4]
#     print(name)
#     #print(yaw)
#     print(score)
#     bbox = BoundingBox3D(center = pos, front = (np.sin(yaw),np.cos(yaw),0), left = (np.cos(yaw),-np.sin(yaw),0) , up = (0,0,1),  size = dim, label_class=name, confidence = score)
#     bbox_list.append(bbox)

# vis = Visualizer()
# lut = LabelLUT()
# for val in sorted(dataset.label_to_names.keys()):
#     lut.add_label(val, val)
#     vis.visualize([{
#         "name": "KITTI",
#         'points': data['point']
#     }], lut, bbox_list)



INFO - 2022-06-13 20:34:21,416 - object_detection - Loading checkpoint /content/pointpillars_kitti_202012221652utc.pth


[[box:65 (class=Pedestrian, conf=1.0), box:66 (class=Pedestrian, conf=1.0), box:67 (class=Pedestrian, conf=1.0), box:68 (class=Pedestrian, conf=1.0), box:69 (class=Pedestrian, conf=1.0), box:70 (class=Pedestrian, conf=0.9999999), box:71 (class=Pedestrian, conf=0.9999938), box:72 (class=Pedestrian, conf=0.99999213), box:73 (class=Pedestrian, conf=0.9998646), box:74 (class=Pedestrian, conf=0.9949215), box:75 (class=Pedestrian, conf=0.99481416), box:76 (class=Pedestrian, conf=0.9946307), box:77 (class=Pedestrian, conf=0.991576), box:78 (class=Pedestrian, conf=0.9894577), box:79 (class=Pedestrian, conf=0.9808529), box:80 (class=Pedestrian, conf=0.80382997), box:81 (class=Pedestrian, conf=0.7479078), box:82 (class=Pedestrian, conf=0.736358), box:83 (class=Pedestrian, conf=0.5878794), box:84 (class=Pedestrian, conf=0.57928765), box:85 (class=Pedestrian, conf=0.494383), box:86 (class=Pedestrian, conf=0.4922297), box:87 (class=Pedestrian, conf=0.4633133), box:88 (class=Pedestrian, conf=0.44989